# Imports

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import zipfile

# Testing of Siamese

In [9]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)  # Input 112 * 112 * 8
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(16, 24, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(24 * 14 * 14, 36)  # Calculating output size after convolutions
        self.fc2 = nn.Linear(36, 28)
        self.fc3 = nn.Linear(28, 16)
        self.fc4 = nn.Linear(16, 1)  # Output should be 1 for binary classification
        self.dropout = nn.Dropout(0.5)

        # Total Params: 175625

    def forward_one(self, x):
        x = F.relu(self.conv1(x))  # input 112*112 * 8
        x = F.max_pool2d(x, 2)  # 56 * 56 * 8
        x = F.relu(self.conv2(x))  # 56 * 56 * 16
        x = F.max_pool2d(x, 2)  # 28 * 28 * 16
        x = F.relu(self.conv3(x))  # 28 * 28 * 24
        x = F.max_pool2d(x, 2)  # 14 * 14 * 24
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x)  # Applying dropout here
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        distance = torch.abs(output1 - output2)
        output = self.fc4(distance)  # Applying sigmoid for binary classification
        return output

In [10]:
# Define the RealTestDataset class
class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        """Read pairs from the file and create a list of image paths and labels."""
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        """Return the total number of pairs."""
        return len(self.pairs)

    def __getitem__(self, idx):
        """Return the images and label for a given index."""
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            print(f"Warning: Image file not found for pair {idx}. Returning None.")
            return None

        if self.transform:
            img1 = self.transform(img1) if img1 else None
            img2 = self.transform(img2) if img2 else None

        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Define the transform for the images
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)

# Filter out pairs with missing images and create DataLoader
filtered_pairs = [pair for pair in test_dataset if pair is not None]
filtered_dataset = [x for x in filtered_pairs if x is not None]
test_loader = DataLoader(filtered_dataset, batch_size=1, shuffle=False)

# Load the saved model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('networks/siamese/final_network.pth'))
model.eval()

# Evaluation function
def evaluate(model, data_loader):
    """Evaluate the model and return mated and non-mated scores."""
    model.eval()
    mated_scores = []
    non_mated_scores = []
    with torch.no_grad():
        for img1, img2, label in data_loader:
            if img1 is None or img2 is None:
                continue
            
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            outputs = model(img1, img2).squeeze()
            score = torch.sigmoid(outputs).item()
            if label.item() == 1:
                mated_scores.append(score)
            else:
                non_mated_scores.append(score)
    
    max_len = max(len(mated_scores), len(non_mated_scores))
    mated_scores.extend([-1] * (max_len - len(mated_scores)))
    non_mated_scores.extend([-1] * (max_len - len(non_mated_scores)))
    
    return mated_scores, non_mated_scores

# Evaluate on the real test set
mated_scores, non_mated_scores = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'scores/siamese/mated_scores.txt'
non_mated_scores_file = 'scores/siamese/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

# Zip the scores
with zipfile.ZipFile('scores/siamese/predictions.zip', 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)
    
# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

# Testing of Triplet

In [11]:
class SiameseNetwork(nn.Module):
    """
    Siamese Network for face recognition.
    """
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)
        self.fc2 = nn.Linear(41, 32)
        self.fc3 = nn.Linear(32, 16)

        # Total params: 198777
        
    def forward_one(self, x):
        """
        Forward pass for one input in the Siamese Network.
        """
        x = F.relu(self.conv1(x))  # input 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # 8 * 56 * 56
        x = F.relu(self.conv2(x))  # 16 * 52 * 52
        x = F.max_pool2d(x, 2)  # 16 * 26 * 26
        x = F.relu(self.conv3(x))  # 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # 32 * 14 * 14
        
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

    def forward(self, input1, input2, input3):
        """
        Forward pass for three inputs in the Siamese Network.
        """
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        output3 = self.forward_one(input3)
        return output1, output2, output3

In [12]:
# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('networks/triplet/network_epoch2.pth'))
model.eval()

# Define the transformation (should match the transformation used during training)
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])

def preprocess_image(image_path):
    """Preprocess the image to match the input format expected by the model."""
    image = Image.open(image_path).convert('L')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def compute_similarity(model, image1, image2, alpha=0.155):
    """Compute the similarity between two images using the trained model."""
    with torch.no_grad():
        embedding1 = model.forward_one(image1)
        embedding2 = model.forward_one(image2)
        distance = torch.mean(F.pairwise_distance(embedding1, embedding2))
        similarity = torch.exp(-alpha * distance)
    return similarity.item()

class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        """Read pairs from the file and create a list of image paths and labels."""
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        """Return the total number of pairs."""
        return len(self.pairs)

    def __getitem__(self, idx):
        """Return the images and label for a given index."""
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            # Handle missing image files by creating placeholder images
            print(f"Warning: Image file not found for pair {idx}. Returning placeholder tensors.")
            img1 = Image.new('L', (112, 112))  # Create a black image placeholder
            img2 = Image.new('L', (112, 112))  # Create a black image placeholder

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

def evaluate(model, data_loader):
    """Evaluate the model and return mated and non-mated scores."""
    model.eval()
    mated_scores = []
    non_mated_scores = []
    all_scores = []
    all_labels = []
    total_pairs = len(data_loader.dataset)

    print(f"Evaluating {total_pairs} pairs...")
    for idx, (img1, img2, label) in enumerate(data_loader):
        if img1 is None or img2 is None:
            mated_scores.append(-1)
            non_mated_scores.append(-1)
            continue
        
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        # Compute similarity
        score = compute_similarity(model, img1, img2)
        
        all_scores.append(score)
        all_labels.append(label.item())

        if label.item() == 1:
            mated_scores.append(score)
        else:
            non_mated_scores.append(score)

        # Print progress
        if (idx + 1) % 100 == 0 or (idx + 1) == total_pairs:
            print(f"Progress: {idx + 1}/{total_pairs}")

    return mated_scores, non_mated_scores, all_scores, all_labels

# Evaluate on the real test set specified in pairs.txt
mated_scores, non_mated_scores, all_scores, all_labels = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'scores/triplet/mated_scores.txt'
non_mated_scores_file = 'scores/triplet/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

# Zip the files
zip_filename = 'scores/triplet/predictions.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)

print(f"Predictions saved to {zip_filename}")

# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")

# Calculate mean absolute error (MAE)
mae = sum(abs(score - label) for score, label in zip(all_scores, all_labels)) / len(all_labels)
print(f"Mean Absolute Error (MAE): {mae:.4f}")


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

# Testing Quadruplet

In [13]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=0)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 12 * 12, 41)
        self.fc2 = nn.Linear(41, 32)
        self.fc3 = nn.Linear(32, 16)
        
        # Total params: 198777

    def forward_one(self, x):
        # Pass through convolutional layers with ReLU and max pooling
        x = F.relu(self.conv1(x))  # output: 8 * 112 * 112
        x = F.max_pool2d(x, 2)  # output: 8 * 56 * 56
        x = F.relu(self.conv2(x))  # output: 16 * 52 * 52
        x = F.max_pool2d(x, 2)  # output: 16 * 26 * 26
        x = F.relu(self.conv3(x))  # output: 32 * 24 * 24
        x = F.max_pool2d(x, 2)  # output: 32 * 12 * 12

        # Flatten the tensor and pass through fully connected layers
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

    def forward(self, input1, input2, input3, input4):
        # Forward pass for all four inputs
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        output3 = self.forward_one(input3)
        output4 = self.forward_one(input4)
        return output1, output2, output3, output4

In [14]:
# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load('networks/quadruplet/network_epoch10.pth'))
model.eval()

# Define the transformation (should match the transformation used during training)
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])

def preprocess_image(image_path):
    """Preprocess the image to match the input format expected by the model."""
    image = Image.open(image_path).convert('L')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def compute_similarity(model, image1, image2, alpha=0.155):
    """Compute the similarity between two images using the trained model."""
    with torch.no_grad():
        embedding1 = model.forward_one(image1)
        embedding2 = model.forward_one(image2)
        distance = torch.mean(F.pairwise_distance(embedding1, embedding2))
        similarity = torch.exp(-alpha * distance)
    return similarity.item()

class RealTestDataset(Dataset):
    def __init__(self, image_folder, pairs_file, transform=None):
        self.image_folder = image_folder
        self.pairs = self._read_pairs(pairs_file)
        self.transform = transform

    def _read_pairs(self, pairs_file):
        """Read pairs from the file and create a list of image paths and labels."""
        pairs = []
        with open(pairs_file, 'r') as file:
            for line in file:
                name_1, number_1, name_2, number_2 = line.strip().split()
                img1_path = os.path.join(self.image_folder, name_1, f'{name_1}_{int(number_1):04d}.png')
                img2_path = os.path.join(self.image_folder, name_2, f'{name_2}_{int(number_2):04d}.png')
                label = 1 if name_1 == name_2 else 0
                pairs.append((img1_path.replace('\\', '/'), img2_path.replace('\\', '/'), label))
        return pairs

    def __len__(self):
        """Return the total number of pairs."""
        return len(self.pairs)

    def __getitem__(self, idx):
        """Return the images and label for a given index."""
        img1_path, img2_path, label = self.pairs[idx]
        try:
            img1 = Image.open(img1_path).convert('L')
            img2 = Image.open(img2_path).convert('L')
        except FileNotFoundError:
            # Handle missing image files by creating placeholder images
            print(f"Warning: Image file not found for pair {idx}. Returning placeholder tensors.")
            img1 = Image.new('L', (112, 112))  # Create a black image placeholder
            img2 = Image.new('L', (112, 112))  # Create a black image placeholder

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor()
])

# Initialize the dataset and dataloader
image_folder = 'lfw_cropped/lfw_cropped'  # Update with the path to your unzipped dataset folder
pairs_file = 'pairs.txt'  # Update with the path to pairs.txt
test_dataset = RealTestDataset(image_folder, pairs_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

def evaluate(model, data_loader):
    """Evaluate the model and return mated and non-mated scores."""
    model.eval()
    mated_scores = []
    non_mated_scores = []
    all_scores = []
    all_labels = []
    total_pairs = len(data_loader.dataset)

    print(f"Evaluating {total_pairs} pairs...")
    for idx, (img1, img2, label) in enumerate(data_loader):
        if img1 is None or img2 is None:
            mated_scores.append(-1)
            non_mated_scores.append(-1)
            continue
        
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        # Compute similarity
        score = compute_similarity(model, img1, img2)
        
        all_scores.append(score)
        all_labels.append(label.item())

        if label.item() == 1:
            mated_scores.append(score)
        else:
            non_mated_scores.append(score)

        # Print progress
        if (idx + 1) % 100 == 0 or (idx + 1) == total_pairs:
            print(f"Progress: {idx + 1}/{total_pairs}")

    return mated_scores, non_mated_scores, all_scores, all_labels

# Evaluate on the real test set specified in pairs.txt
mated_scores, non_mated_scores, all_scores, all_labels = evaluate(model, test_loader)

# Save mated and non-mated scores to .txt files
mated_scores_file = 'scores/quadruplet/mated_scores.txt'
non_mated_scores_file = 'scores/quadruplet/non_mated_scores.txt'

with open(mated_scores_file, 'w') as f:
    for score in mated_scores:
        f.write(f'{score}\n')

with open(non_mated_scores_file, 'w') as f:
    for score in non_mated_scores:
        f.write(f'{score}\n')

print(f'Mated Scores: {mated_scores[:5]}')
print(f'Non-Mated Scores: {non_mated_scores[:5]}')

# Zip the files
zip_filename = 'scores/quadruplet/predictions.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(mated_scores_file)
    zipf.write(non_mated_scores_file)

print(f"Predictions saved to {zip_filename}")

# Calculate accuracy for mated scores (> 0.5)
mated_correct = sum(score > 0.5 for score in mated_scores)
mated_accuracy = mated_correct / len(mated_scores) * 100

# Calculate accuracy for non-mated scores (< 0.5)
non_mated_correct = sum(score < 0.5 for score in non_mated_scores)
non_mated_accuracy = non_mated_correct / len(non_mated_scores) * 100

print(f"Mated Accuracy: {mated_accuracy:.2f}%")
print(f"Non-Mated Accuracy: {non_mated_accuracy:.2f}%")

# Calculate mean absolute error (MAE)
mae = sum(abs(score - label) for score, label in zip(all_scores, all_labels)) / len(all_labels)
print(f"Mean Absolute Error (MAE): {mae:.4f}")


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.